In [7]:
import os
import sys

import pandas as pd
import numpy as np
import scipy as sp

pd.set_option('display.max_columns', None)

from quasinet.qnet import Qnet, save_qnet

In [2]:
PRESENCE_DF = pd.read_csv("example_data/ICD_PRESENCE_DF.csv")
PRESENCE_DF.sample(2)

,patient_id,target,Z00,E78,I10,Z23,M54,M25,J01,M79,R10,Z12,R53,R07,J06,J30,G47,J02,R05,K21,E66,Z13,R06,Z79,F41,M99,E11,J20,Z01,Z68,D23,E55,R51,M62,M51,R73,F17,D48,L03,J45,J32,F32,L25,R19,E29,R94,H10,R11,K64,B35,E03,R03,R00,F43,Z11,R42,M47,N52,Z51,R50,J34,M77,L82,H66,M75,S33,R20,G89,R68,D12,Z71,M60,L57,H61,Z87,K62,N40,K29,K57,R21,R78,M19,M72,K76,N20,L72,J40,K52,H52,B07,R35,R31,Z98,D64,M10,N39,M50,H40,L81,F90,R09,I49
1307,P27569056501,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1016,P02886249502,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## **Generate qnet from parts**

In [3]:
base_columns = ['patient_id', 'target']
TOTAL = PRESENCE_DF.drop(base_columns, 1)
TARGET = PRESENCE_DF['target']
TOTAL['XXXX'] = list(TARGET)
X = TOTAL.values.astype(str)
print(X.shape)

(10000, 101)


In [5]:
#%%time
ALL_COLS = list(TOTAL.columns)
PARTIBUS_QNET = []

# create a list of indices that divide the number of rows into $N_parts equal subsets
N_parts = 30
indices = np.linspace(0, 
                      TOTAL.shape[1], 
                      num = N_parts + 1, 
                      dtype = int)
idx_subsets = [[i for i in range(indices[i], indices[i+1])] for i in range(len(indices)-1)]

X = TOTAL.values.astype(str)
print(X.shape)

for i, idx_subset in enumerate(idx_subsets):
    print("Training trees - %d to %d" % (idx_subset[0], idx_subset[-1]))

    PARS_QNET = Qnet(
                feature_names = list(TOTAL.columns),
                alpha = 0.1,
                n_jobs = -1
    )
    PARS_QNET.fit(X, index_array = idx_subset)
    
    if not PARTIBUS_QNET:
        PARTIBUS_QNET = PARS_QNET
    else:
        PARTIBUS_QNET.mix(PARS_QNET, list(TOTAL.columns))

(10000, 101)
Training trees - 0 to 2
Training trees - 3 to 5
Training trees - 6 to 9
Training trees - 10 to 12
Training trees - 13 to 15
Training trees - 16 to 19
Training trees - 20 to 22
Training trees - 23 to 25
Training trees - 26 to 29
Training trees - 30 to 32
Training trees - 33 to 36
Training trees - 37 to 39
Training trees - 40 to 42
Training trees - 43 to 46
Training trees - 47 to 49
Training trees - 50 to 52
Training trees - 53 to 56
Training trees - 57 to 59
Training trees - 60 to 62
Training trees - 63 to 66
Training trees - 67 to 69
Training trees - 70 to 73
Training trees - 74 to 76
Training trees - 77 to 79
Training trees - 80 to 83
Training trees - 84 to 86
Training trees - 87 to 89
Training trees - 90 to 93
Training trees - 94 to 96
Training trees - 97 to 100


In [9]:
save_qnet(PARTIBUS_QNET, "EXAMPLE_QNET.joblib")